In [ ]:
dt='20211217'

In [ ]:
from datetime import datetime, timedelta

c_dt = datetime.strptime(dt, '%Y%m%d')

c_dt = c_dt + timedelta(days=-1)
o_dt = c_dt + timedelta(days=-1)

c_dt = datetime.strftime(c_dt, '%Y%m%d')
o_dt = datetime.strftime(o_dt, '%Y%m%d')

print('c_dt : %s' % c_dt)
print('o_dt : %s' % o_dt)

In [ ]:
import hvac
import urllib3

def get_secrets(path):
    # Warning 제거
    urllib3.disable_warnings()

    secrets = hvac.Client(verify=False).secrets.kv.v2.read_secret_version(path=path)["data"]["data"]
    return secrets

from pyspark.sql import SparkSession
from pyspark import SparkConf

redis_info = get_secrets(path="unhash-redis-cluster")

sparkConf = SparkConf()
sparkConf.set("spark.jars", "hdfs:///jars/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar")

sparkConf.set("spark.redis.host", redis_info['host'])
sparkConf.set("spark.redis.port", redis_info['port'])
sparkConf.set("spark.redis.auth", redis_info['password'])
sparkConf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

spark = SparkSession.builder.config(conf=sparkConf).enableHiveSupport().getOrCreate()

In [ ]:
spark.sql(f""" select ye_hashed, raw from aidp.svc_mgmt_num_mapping where dt={c_dt} """).registerTempTable('latest_table')
spark.sql(f""" select ye_hashed, raw from aidp.svc_mgmt_num_mapping where dt={o_dt} """).registerTempTable('old_table')

In [ ]:
df = spark.sql("""

    select
        l.ye_hashed,
        l.raw
    from 
        latest_table l left outer join old_table o
        on l.raw = o.raw
    where
        o.raw is null
        and l.ye_hashed is not null

""")

In [ ]:
if df.count() > 0:
    df.write.format("org.apache.spark.sql.redis").option("table", "unhash").option("key.column", "ye_hashed").save(mode="append")

In [ ]:
spark.stop()